In [70]:
import pandas as pd
import numpy as np

In [47]:
def del_escape(text):
    char_ls=list(text)
    reduced_ls=[char for char in char_ls if "{c!r}".format(c=char).replace("'","")==char]
    return "".join(reduced_ls)

In [51]:
data=pd.read_json("willcrawl_1.json")[["inner_id","insta_id","content"]]
data=data.dropna()
data["content"]=data["content"].apply(del_escape)
data.pivot_table(index="insta_id",values=["content"],aggfunc="sum").reset_index().to_csv("community.csv",index=False)

In [19]:
df=pd.read_json("./insta_crawl_follow/30_follow.json", encoding="utf8")
df

,start,end,relation_type,shortcode
0,6415045405,8929079837,follow,
1,6415045405,7925828783,follow,
2,6415045405,5534279700,follow,
3,7804368230,10007428293,follow,
4,1354494015,1392967776,follow,
...,...,...,...,...
301512,1421862991,329669090,follow,
301513,35084766855,4638973097,follow,
301514,9163554229,13128310191,follow,
301515,1771827841,2284514001,follow,


In [20]:
from collections import defaultdict
graph=defaultdict(set)
for i,inner_id in enumerate(df["start"]):
    graph[inner_id].add(df["end"][i])
    graph[df["end"][i]].add(inner_id)
graph

defaultdict(set,
            {6415045405: {33806425,
              179212722,
              318994122,
              1177762925,
              1268977795,
              1390359282,
              1450095726,
              1484401049,
              1487811563,
              1509024140,
              1525616918,
              1544674104,
              1549982085,
              1644152902,
              1714895907,
              1742816996,
              1801837242,
              1806680470,
              1813208836,
              2086628085,
              2105489240,
              2190052217,
              2870781777,
              3016334244,
              3168017502,
              3249552381,
              3294367057,
              3311674601,
              3554769900,
              3773384036,
              3889699596,
              3924851094,
              4904913630,
              5460512049,
              5534279700,
              5603223762,
              5776624290,
             

In [21]:
def bfs(begin,graph,threshold):
    queue=[(-1,begin)]
    visited=set()
    while queue:
        n=queue.pop(0)
        if len(visited)==0:
            visited.add(n)
            queue+=[(n[1],x) for x in graph[n[1]]]
        else:
            if n[1] not in list(zip(*visited))[1]:
                visited.add(n)
                queue+=[(n[1],x) for x in graph[n[1]]-set(list(zip(*visited))[1])]
        if len(visited)>=threshold:
            break
    return visited

In [59]:
community_ls=list(bfs(1644152902,graph,100)-{(-1,1644152902)})


In [41]:
pd.Series(list(set(list(zip(*community_ls))[0]+list(zip(*community_ls))[1]))).to_csv("willcrawl_1.csv",index=False)

In [60]:
dic=defaultdict(int)
i=0
for inner_id in list(zip(*community_ls))[0]:
    if inner_id not in dic:
        dic[inner_id]=i
        i+=1
for inner_id in list(zip(*community_ls))[1]:
    if inner_id not in dic:
        dic[inner_id]=i
        i+=1    
dic

defaultdict(int,
            {6415045405: 0,
             1644152902: 1,
             3294367057: 2,
             2870781777: 3,
             9265106119: 4,
             21402699645: 5,
             3168017502: 6,
             37442516662: 7,
             7615378913: 8,
             34815598243: 9,
             33806425: 10,
             19864538426: 11,
             40678797776: 12,
             29189130267: 13,
             8496576989: 14,
             3311674601: 15,
             318994122: 16,
             20691495264: 17,
             1525616918: 18,
             1509024140: 19,
             1177762925: 20,
             1742816996: 21,
             33869897370: 22,
             13209073244: 23,
             9454425330: 24,
             1390359282: 25,
             34135934419: 26,
             33547825675: 27,
             13740350700: 28,
             3554769900: 29,
             41012755149: 30,
             1544674104: 31,
             6211142394: 32,
             40326679184: 

In [61]:
with open("sample.txt","w",encoding="utf-8") as f:
    f.write("\n".join([" ".join([str(dic[x[0]]),str(dic[x[1]])])for x in community_ls]))

In [62]:
import snap

In [144]:
G1=snap.LoadEdgeList(snap.PUNGraph,"sample.txt",0,1," ")
CmtyV = snap.TCnComV()
modularity=snap.CommunityCNM(G1,CmtyV)

In [64]:
import pickle

In [85]:
with open("pred_ls.pickle","rb") as f:
    file=pickle.load(f)

In [148]:
(np.array(file).T[1]/np.array(file).T[1].sum(axis=1).reshape(-1,1))[33]

array([0.89849971, 0.10150029])

In [88]:
(np.array(file).T[1]/np.array(file).T[1].sum(axis=1).reshape(-1,1)).argmax(axis=1)

array([0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0], dtype=int64)

In [102]:
community_df=pd.read_csv("community.csv")
community_df["prediction"]=(np.array(file).T[1]/np.array(file).T[1].sum(axis=1).reshape(-1,1)).argmax(axis=1)
# target user 제거
community_df=community_df.drop(33)
community_df

,insta_id,content,prediction
0,163kg__,"내 주변엔 좋은사람들만 있어서 행복해막내동생이랑🤪내 코 언제 나아,,,초딩아님‼️1...",0
1,97s_ba_e,마니 컸다..세탁실 앞에서 자는 그는..하루종일 잔다#사기 #보이스피싱 조심하세요...,0
2,again._.ty,#다시 #짧은 #글귀 #글 #시 #데일리#사진 #빛바랜 #추억 #칠하기#팔로우좋아요...,0
3,anyeji620,성남시 청년지원센터해피봄스데이🥳🥳.청년이봄 1주년 축하드립니다><코로나19로 인해 ...,0
4,clothing.wholesaler,동대문~땡!ㆍ도매언니/밤시장/ 밴드로 초대합니다.아래를 클릭하시면 밴드 가입 가능하...,1
5,damun_saju,"🔎후기🔎마음이 편해지는 사주, 담운사주입니다!이 시국에 나가는 게 꺼려진다면 집...",1
6,dan_ggyu,김떡순 먹으러 가다가 먹고와서 일 할 생각하니 다리에 힘풀려서 길바닥 안부인사 묻고...,0
7,desenhos_da_rapaziada,Arrasta pro lado para ver o progresso do desen...,1
8,dgz_drawings,Izuku Midorya #desenhosanimes #anime #desenhos...,1
9,dongbo1608,대학수시(학교장추천) 가능여부로어제 오늘아침까지 전화기에 불났다불안한 맘은 알지만 ...,0


In [110]:
community_df["insta_id"].tolist()

['163kg__',
 '97s_ba_e',
 'again._.ty',
 'anyeji620',
 'clothing.wholesaler',
 'damun_saju',
 'dan_ggyu',
 'desenhos_da_rapaziada',
 'dgz_drawings',
 'dongbo1608',
 'gangmyeongji5936',
 'ha_rimi2',
 'honey._.lim',
 'hyeoni_nyang_',
 'its.pusaaaaaa',
 'j_hye0920',
 'je.kt.db',
 'ju__hee2',
 'jwithsecret',
 'lblanc1984',
 'li.rva',
 'luv__rin2',
 'me_woon_gasinae',
 'queentarot_',
 'rahel_ing',
 'sabina.tarot',
 'soheei_0101',
 'son___b',
 'suyeon2da_',
 'tarot3860',
 'tarotmaster.bg',
 'tarotmaster_nora',
 'tr_k_hoon',
 'unknown__074',
 'viciadoanimes',
 'ye__._.__on',
 'yen._.o',
 'young._.ei',
 'younghyeji']

In [130]:
4904913630 in inner_insta_dic

False

In [117]:
inner_insta_dic=dict(zip(pd.read_json("willcrawl_username_1.json")["inner_id"],pd.read_json("willcrawl_username_1.json")["username"]))
community_insta_dic={}
for x in inner_insta_dic:
    community_insta_dic[dic[x]]=inner_insta_dic[x]
community_insta_dic

{37: 'rahel_ing',
 0: 'tarot3860',
 13: 'ddddddd_778_u',
 90: 'queentarot_',
 95: 'rickabreu13',
 27: 'dgz_drawings',
 89: 'jwithsecret',
 58: 'yen._.o',
 10: 'dasolzzang',
 52: 'zzz__ing',
 87: '2x_marie_x2',
 67: 'lblanc1984',
 46: 'wannagobak',
 1: 'uk_jae98',
 49: '163kg__',
 23: 'sueliee3',
 6: 'son___b',
 20: 'nijey_j',
 38: 'jjo_er',
 55: 'younghyeji',
 59: 'jelyjely_daily',
 60: '97s_ba_e',
 35: '0kyu_ri0',
 91: 'gangmyeongji5936',
 92: 'damun_saju',
 75: 'j_hye0920',
 33: 'dowoq9q',
 99: 'ash_ley_04',
 43: 'wijnmmml',
 22: 'hyeoni_nyang_',
 77: 'sullofursivee01__',
 9: 'li.rva',
 83: 'suyeon2da_',
 80: 'again._.ty',
 7: 'kimmisu1209',
 64: 'ye__._.__on',
 63: 'tr_k_hoon',
 69: 'anyeji620',
 70: 'clothing.wholesaler',
 16: 'hello_yunseo',
 4: 'me_woon_gasinae',
 30: 'chaeeun.__.05',
 94: 'dongbo1608',
 21: 'dauuuumy',
 81: '_eunbis',
 93: 'tarotmaster.bg',
 15: 'hyejin_loves',
 28: 'ov_._._o',
 25: 'jiihyuney',
 24: 'ha_rimi2',
 39: 'ejghiejghi',
 32: 'its.pusaaaaaa',
 40: 'vic

In [114]:
insta_label_dic=dict(zip(community_df["insta_id"],community_df["prediction"]))
insta_label_dic

{'163kg__': 0,
 '97s_ba_e': 0,
 'again._.ty': 0,
 'anyeji620': 0,
 'clothing.wholesaler': 1,
 'damun_saju': 1,
 'dan_ggyu': 0,
 'desenhos_da_rapaziada': 1,
 'dgz_drawings': 1,
 'dongbo1608': 0,
 'gangmyeongji5936': 0,
 'ha_rimi2': 0,
 'honey._.lim': 0,
 'hyeoni_nyang_': 1,
 'its.pusaaaaaa': 1,
 'j_hye0920': 0,
 'je.kt.db': 0,
 'ju__hee2': 1,
 'jwithsecret': 0,
 'lblanc1984': 0,
 'li.rva': 1,
 'luv__rin2': 1,
 'me_woon_gasinae': 1,
 'queentarot_': 1,
 'rahel_ing': 1,
 'sabina.tarot': 0,
 'soheei_0101': 1,
 'son___b': 0,
 'suyeon2da_': 0,
 'tarot3860': 0,
 'tarotmaster.bg': 0,
 'tarotmaster_nora': 0,
 'tr_k_hoon': 0,
 'unknown__074': 0,
 'viciadoanimes': 1,
 'ye__._.__on': 1,
 'yen._.o': 1,
 'young._.ei': 0,
 'younghyeji': 0}

In [134]:
from collections import Counter
total=[]
for Cmty in CmtyV:
    total.append(Counter([insta_label_dic[community_insta_dic[x]] for x in list(Cmty) if (x in community_insta_dic) and (community_insta_dic[x] in insta_label_dic)]))
total  

[Counter({0: 23, 1: 16})]

In [143]:
result=np.array([[x[0]/(x[0]+x[1]),x[1]/(x[0]+x[1])]for x in total]).max(axis=0)/np.array([[x[0]/(x[0]+x[1]),x[1]/(x[0]+x[1])]for x in total]).max(axis=0).sum()
result

array([0.58974359, 0.41025641])